# Media

**scikit-rf** supports some basic device simulation based on transmission line models, such as coaxial lines, hollow waveguides, microstrip-lines, freespace, etc.

* Network creation is accomplished through the `Media` class and subclasses, which represent a transmission line object for a given medium.
* Once constructed, a `Media` object contains properties/methods such as:
  * `propagation constant`
  * `characteristic impedance`
  * `line`, `short`, `open`, `resistor`, `inductor`, etc...
- The list of supported media is in the documentation Media API page.

In [ ]:
import skrf as rf
from skrf.media import CPW  # Co-Planar Waveguide 
rf.stylely()

## Coaxial Example
- Simulating a VNA measurement¶ of a RG-58 cable
<img src="figures/media_coax_measurement.svg" width="600"/>

- A `Frequency` object is the minimum requirement to construct a Media object. Other parameters depend of the medium type.
- The port impedance `z0_port` can be passed to renormalize the media characteristic impedance (default) `z0` to the port impedance of a simulated measurement. 

In [ ]:
from skrf.media import Coaxial
freq = rf.Frequency(start=1, stop=2, npoints=1001, unit='GHz')

# media with z0_port the port impedance of the VNA
# https://en.wikipedia.org/wiki/Coaxial_cable#Standards
rg58 = Coaxial(freq, Dint=0.91e-3, Dout=2.95e-3, epsilon_r=2.3, z0_port=50)
print(rg58)

Let’s create a line network corresponding to a 1m length of coax:


In [ ]:
line = rg58.line(d=1, unit='m', name='rg58_1m')

The effect of the mismatch between the port impedance `z0_port` and the characteristic impedance `z0` of the line defined by the geometry (NB: can also be forced to an arbitrary value) is observed on RL:

In [ ]:
line.plot_s_db(m=0, n=0)

## Example: Stub Design Optimization
In this example, scikit-rf is used to optimize a single stub impedance matching network design. 

In [ ]:
from scipy.optimize import fmin
from skrf.media import CPW
freq = rf.Frequency(75, 110, 101, 'GHz')
cpw = CPW(freq, w = 10e-6, s = 5e-6, ep_r = 10.6, z0_port = 50)

# the load we are trying to match
load = cpw.load(.2+.2j)

- First, we create a "cost" function that returns something we want to minimize, such as the reflection coefficient magnitude at the band center. 

In [ ]:
# single stub circuit generator function
def shunt_stub(med, d0, d1):
    return med.line(d0, unit='deg') ** med.shunt_delay_open(d1, unit='deg')

- Then, one of scipy’s minimization algorithms is used to determine the optimal parameters of the stub lengths to minimize this cost.


In [ ]:
# define the cost function we want to minimize (this uses sloppy namespace)
def cost(d):
    # prevent negative length lines, returning high cost
    if d[0] < 0 or d[1] < 0:
        return 1e3
    return (shunt_stub(cpw, d[0], d[1]) ** load)[100].s_mag.squeeze()

In [ ]:
# initial guess of optimal delay lengths in degrees
d0 = 120,40 # initial guess

#determine the optimal delays
d_opt = fmin(cost, (120, 40))

d_opt

## Rectangular Waveguide
Let's create a 30cm long WR-10 rectangular waveguide:

<img src="figures/media_rectangularwaveguide.svg" width="200" align="right"/>

In [ ]:
from skrf.media import RectangularWaveguide
freq = rf.Frequency(75, 110, 1001, 'GHz')
wr10  = RectangularWaveguide(freq, a=rf.to_meters(100, 'mil'), b=rf.to_meters(50, 'mil'),
                             ep_r=1.0,  rho=1.68e-08)
print(wr10)

Line creation:

In [ ]:
wr10_100 = wr10.line(100e-3, unit='m', name='wr10 100mm', z0=50)
print(wr10_100)

In [ ]:
wr10_100.plot_s_mag(m=1, n=0)

## Example: Measured-like WR-10 waveguide
- VNA measured S-Parameters of a hollow waveguide.
- The VNA has a known port impedance (usually 50 Ohm) and coaxial connectivity.
- A coaxial to waveguide transitions are used. The transition is calibrated at the waveguide interface. Thus, VNA port impedance override the characteristic impedance of the waveguide.

- Measurement will store the port impedance instead of the characteristic impedance. 
- This is not normalization: the actual characteristic impedance of the line is not measured.


<img src="figures/media_waveguide_measurement.svg" width="600"/>

To get an S-Parameters network with 50-Ohm port impedance, either the port impedance `z0_override` can be specified at media object construction (NB: impedance can also be overridden manually).

In [ ]:
# port impedance at media construction method
wr10_meas  =  RectangularWaveguide(freq, a=rf.to_meters(100, 'mil'), b=rf.to_meters(50, 'mil'),
                                   ep_r=1.0,  rho=1.68e-08,
                                   z0_override=50)
wr10_100_measured = wr10_meas.line(100e-3, unit='m', name='wr10 100mm z0_port')
print(wr10_100_measured2)

In [ ]:
wr10_100_measured.plot_s_mag(m=0, n=0)  # matched

## Slab of  Si in Freespace

A plane-wave in freespace from 10-20GHz.

In [ ]:
from skrf.media import Freespace

freq = rf.Frequency(10,20,101,'ghz')
air =  Freespace(freq)
air

In [ ]:
air.z0[:2] # 377ohm baby!

A plane wave in Si:

In [ ]:
si = rf.Freespace(freq, ep_r = 11.2)
si.z0[:3] # ~110ohm


Simulate a 1cm slab of Si in half-space,

In [ ]:
slab = air.thru() ** si.line(1, 'cm') ** air.thru()
slab.plot_s_db(n=0) 